In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from numpy import mean

In [2]:
os.chdir(r'\Users\hossa084\Downloads\wildfire\WildFires-master')

In [ ]:

#creating a function to download a zip and unzip it in my desired location
def download_data(dw_url,destination):
    import wget
    import zipfile
    
    filename = wget.download(dw_url)
    print("Successfully downloaded", filename)
    
    with zipfile.ZipFile(destination + "/" + filename, 'r') as zip_ref:
        zip_ref.extractall(destination)

In [ ]:
#using my function
download_data('https://github.com/ouladsayadyounes/WildFires/archive/refs/heads/master.zip',r'\Users\hossa084\Downloads\wildfire\WildFires-master')

In [3]:
#opening the CSV and viewing part of it
wildfires = pd.read_csv('WildFires_DataSet.csv', float_precision='high')
wildfires.head()

,NDVI,LST,BURNED_AREA,CLASS
0,0.506782,14584.272727,4.692308,no_fire
1,0.522150,14655.833333,5.000000,no_fire
2,0.682284,14780.000000,5.000000,fire
3,0.120046,13298.500000,3.500000,no_fire
4,0.568734,14743.000000,5.000000,no_fire


In [4]:
#ML does not like classes that are a string
#change the classes to numbers
wildfires['CLASS'] = wildfires['CLASS'].map({'fire': 1, 'no_fire': 0})

In [5]:
#seperating the features from the labels
wildfire_features = wildfires.copy()
wildfire_labels = wildfire_features.pop('CLASS')
wildfire_labels.head()



0    0
1    0
2    1
3    0
4    0
Name: CLASS, dtype: int64

In [6]:
#create functions to normalize the data for the Neural Net
def normalize(x):
    x_med = x.median()
    x_std = x.std()
    x = ((x-x_med)/x_std)
    return x

def normalize_dataframe(pandas_df):
    pandas_df = pandas_df.apply(normalize)
    
    return pandas_df

In [7]:
wildfire_features = normalize_dataframe(wildfire_features) 
wildfire_features.head()
#now you might be thinking to yourself, "That's shome pretty shweet data"

,NDVI,LST,BURNED_AREA
0,-0.628978,-0.167546,-0.391111
1,-0.463460,0.027480,0.135948
2,1.261242,0.365876,0.135948
3,-4.794266,-3.671703,-2.433466
4,0.038267,0.265038,0.135948


In [8]:

#changing this from a series to values
wildfire_labels.values


array([0, 0, 1, ..., 1, 0, 1], dtype=int64)

In [9]:
wildfire_features = np.array(wildfire_features)
wildfire_features

array([[-6.28978449e-01, -1.67545939e-01, -3.91111091e-01],
       [-4.63459505e-01,  2.74804235e-02,  1.35947705e-01],
       [ 1.26124186e+00,  3.65875733e-01,  1.35947705e-01],
       ...,
       [ 5.15028005e-01,  2.88203790e-01, -3.28993790e+00],
       [ 5.30871209e-01, -2.82003661e+00,  7.16010091e-04],
       [-1.68922379e-01, -1.02464888e+00,  1.35947705e-01]])

In [10]:
#splitting my features and labels into train and test
wf_features_train, wf_features_test, wf_class_train, wf_class_test = model_selection.train_test_split(wildfire_features, wildfire_labels, test_size=0.3)

In [11]:
# Support Vector Machine
SupportVectorMachine = svm.SVC()
SupportVectorMachine.fit(wf_features_train, wf_class_train)

SVC()

In [12]:
y_pred = SupportVectorMachine.predict(wf_features_test)
y_pred[:10],wf_class_test[:10]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 1310    0
 370     0
 1586    0
 1081    0
 588     0
 837     0
 609     0
 68      0
 1556    0
 305     0
 Name: CLASS, dtype: int64)

In [ ]:
# for i in y_pred:
#     if i > .5:
#         print("Fire")
#     else:
#         print("No fire")

In [13]:
# Predict
print(SupportVectorMachine.predict([[1.27242,0.385876,0.134487]]))
print(SupportVectorMachine.predict([[1.461242,0.30876,0.135123]]))
print(SupportVectorMachine.predict([[1.261242,0.365876,0.135948]]))
print(SupportVectorMachine.predict([[1.461242,0.30876,0.13548]]))
print(SupportVectorMachine.predict([[1.261242,0.365876,0.135948]]))
print(SupportVectorMachine.predict([[1.261242,0.365876,0.135948]]))

[0]
[0]
[0]
[0]
[0]
[0]


In [14]:
from sklearn.metrics import classification_report
# fit a SVM model to the data
SupportVectorMachine = SVC()
SupportVectorMachine.fit(wildfire_features, wildfire_labels)
print(SupportVectorMachine)
# make predictions
expected = wildfire_labels
predicted = SupportVectorMachine.predict(wildfire_features)
# summarize the fit of the model
print(classification_report(expected, predicted))


SVC()
              precision    recall  f1-score   support

           0       0.79      0.99      0.88      1327
           1       0.77      0.08      0.14       386

    accuracy                           0.79      1713
   macro avg       0.78      0.54      0.51      1713
weighted avg       0.78      0.79      0.71      1713



In [16]:
from numpy import mean
from numpy import std

# define evaluation procedure 

SupportVectorMachine = svm.SVC(gamma='scale')
SupportVectorMachine.fit(wf_features_train, wf_class_train)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(SupportVectorMachine, wf_features_test, wf_class_test, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.783 (0.006)


In [ ]:
# get number of support vectors for each class
clf.n_support_

In [ ]:
# get support vectors
clf.support_vectors_

In [ ]:
# get indices of support vectors
clf.support_